# 11/05 - Pair Programming Limpieza 6 - Guadalupe & Lara

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder 


In [2]:
df = pd.read_csv("attacks_limpiando_con_coordenadas.csv", index_col = 0)

En la sesión de hoy trabajaremos con la codificación de las variables categóricas. Recordemos que esto es esencial ya que muchos modelos de machine learning no trabajan con variables numéricas, y debemos convertirlas a variables numéricas.

Ninguna de nuestras columnas categóricas en este dataframe parece tener un orden por lo que descartamos usar la codificación de tipo Ordinal Encoding. Trabajaremos, por lo tanto, con codificación de tipo **One Hot Encoding y Label Encoding**.

Lo primero que debemos hacer es identificar cuáles de nuestras columnas es categóricas. Extraed el nombre de las columnas que contienen variables categóricas.


In [3]:
df_cat = df.select_dtypes(include = "object")
columnas_cat = df_cat.columns
columnas_cat 

Index(['type', 'country', 'species_', 'fecha_limpia', 'fatal', 'sex',
       'latitud', 'longitud'],
      dtype='object')

Si nos fijamos en la columna de country es my complicado codificar esa columna ya que hay muchísimas categorías. Para ello, vamos a usar una librería que llamada geopy que nos devuelve las coordenadas de un pais, ciudad, pueblo, lo que imaginemos! El objetivo de esto es poder clasificar nuestros paises en Hemisferio Norte y Sur para poder hacer una codificación más eficaz. Os dejamos por aquí la documentación de la librería para que veais como funciona y el código:

In [4]:
#def extraer_coordenadas(x):
#   try:
#        geolocator = Nominatim(user_agent = "ana")
#        location = geolocator.geocode(x)
#        return location.lat, location.long
#    except:
#        return "no hay datos" 

Nuestra recomendación... no la ejecuteis, tarda muchísimo en aplicarse a nuestro dataframe. Os dejamos el dataframe con las coordenadas al final de la explicación de la sesión.

**2. Codificar "country"**
- Como ya tenemos las coordenadas, deberéis codificar la columna en función de si pertenecen al Hemisferio Norte (HN) o Sur (HS). Aquellas que pertencezcan al HN le pondremos un 1 y a las del HS le pondremos un 0.
- Nota 📌  Perteneceran al HN las que su latitud sea positiva y las del HS las que su latitud sea negativa.



In [5]:
negativos = df["latitud"].str.contains("-")

In [17]:
# Primero creamos una nueva columna llamada "hemisferio" con pandas, para dividir en función de HS o HN
# Después, codificamos la columna con label_encoding para que HS = 0, y HN = 1

patron_neg = r".*\-[0-9].*"
patron_pos = r".*[0-9].*"
 
def hemispherificator(col):
    try:
        if re.findall(patron_neg, col):
            return "HS"
        elif re.findall(patron_pos, col):
            return "HN"
        else:
            return np.nan
    except:
        np.nan

df["Hemisferio"] = df["latitud"].apply(hemispherificator)
df

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,HN
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,HN
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882,HN
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755,HS
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.6585116,-102.0077097,HN
...,...,...,...,...,...,...,...,...,...,...,...
6213,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.7761086,134.755,HS
6214,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.7761086,134.755,HS
6215,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.7761086,134.755,HS
6216,0,Unprovoked,panama,27.0,Unespecific,Unknown,Y,M,8.559559,-81.1308434,HN


[ICONOS]: Hemos optado por convertir los valores de "latitud" que no eran "correctos" (dígitos, con o sin el "-") en np.nan. 
Entonces, al hacer el encoding tendremos 3 valores numéricos (0, 1 y 2). En un primer intento hacemos "label encoding" pero los valores que otorga a HM y HN, al ser aleatorios, no son los que queremos por el enunciado (vaya, que nos pone HS = 1 y HN = 0 y lo quermeos alrevés). Entonces, lo cambiamos por un map para poner en el diccionario el valor a nuestro gusto.

In [27]:
mapa = {"HN": 1, "HS" : 0, np.nan : np.nan} 

In [31]:
df["Hemisferio_encoded"] = df["Hemisferio"].map(mapa)
df.head(10)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,Hemisferio,Hemisferio_encoded
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.7837304,-100.445882,HN,1.0
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.7837304,-100.445882,HN,1.0
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.7837304,-100.445882,HN,1.0
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755,HS,0.0
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.6585116,-102.0077097,HN,1.0
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.7761086,134.755,HS,0.0
6,2018,Unprovoked,brazil,18.0,Tiger shark,Jun,Y,M,-10.3333333,-53.2,HS,0.0
7,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,39.7837304,-100.445882,HN,1.0
8,2018,Unprovoked,usa,15.0,Lemon shark,May,N,M,39.7837304,-100.445882,HN,1.0
9,2018,Unprovoked,usa,12.0,Unespecific,May,N,M,39.7837304,-100.445882,HN,1.0


In [32]:
df["Hemisferio"].unique()

array(['HN', 'HS', nan], dtype=object)

**3. Aplicad un get_dummies a las columnas de fatal y age.**


In [33]:
df.columns

Index(['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'latitud', 'longitud', 'Hemisferio', 'Hemisferio_encoded'],
      dtype='object')

In [24]:
df[["fatal", "age"]]

,fatal,age
0,N,57.0
1,N,11.0
2,N,48.0
3,N,27.0
4,N,27.0
...,...,...
6213,Y,27.0
6214,Y,27.0
6215,Y,27.0
6216,Y,27.0


**4. Aplicad un Label Encoding a la columna species , fecha y type.**


In [ ]:
# Aplicamos el LABEL ENCODING a "Hemisferio" en un dataframe nuevo (encoded)

label_encoded_df = df.copy()

le = LabelEncoder()

label_encoded_df["Hemisferio"]=le.fit_transform(label_encoded_df["Hemisferio"])


**5. Guardad los cambios en un csv nuevo para seguir trabajando con él en siguientes ejercicios de pair.**